# МатМод

### Задание 1

In [1]:
import random
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

gen_count = 1_000_000

def get_event_frequency_function(event_f):
    def event_frequency(**kwargs):
        sample_set = [event_f(**kwargs) for _ in range(gen_count)]
        return sum(sample_set) / gen_count
    return event_frequency

In [2]:
def generate_simple_event(**kwargs):
    p = kwargs['p']
    return random.random() < p

simple_event_frequency_function = get_event_frequency_function(generate_simple_event)

In [3]:
interact_manual(simple_event_frequency_function,
         p=widgets.FloatSlider(min=0, max=1,step=0.01,value=0.5))

interactive(children=(FloatSlider(value=0.5, description='p', max=1.0, step=0.01), Button(description='Run Int…

<function __main__.get_event_frequency_function.<locals>.event_frequency(**kwargs)>

### Задание 2

In [4]:
def handle_slider_change(change):
    values = [simple_event_frequency_function(p=p.value) for p in p_list]
    caption.value = (
        f'The results are {values}'
    )

n_p_list = 3
p_list = [
    widgets.FloatSlider(min=0, max=1, value=0.5, step=0.01, orientation='vertical')
    for i in range(n_p_list)
]

# register slides
for widget in p_list:
    widget.observe(handle_slider_change, names='value')

# display ui
ui = widgets.HBox(p_list)
caption = widgets.Label(value=f'The results are {[simple_event_frequency_function(p=p.value) for p in p_list]}')
display(ui, caption)

Label(value='The results are [0.50062, 0.499669, 0.499743]')

### Задание 3

In [5]:
def get_events_frequencies_function(event_f):
    def events_frequencies(**kwargs):
        sample_set = [event_f(**kwargs) for _ in range(gen_count)]
        return (sum([s[0] and s[1] for s in sample_set]) / gen_count,
                sum([s[0] and not s[1] for s in sample_set]) / gen_count,
                sum([not s[0] and s[1] for s in sample_set]) / gen_count,
                sum([not s[0] and not s[1] for s in sample_set]) / gen_count)
    return events_frequencies

def generate_complex_event(**kwargs):
    p_a, p_ba = kwargs['p_a'], kwargs['p_ba']
    A = generate_simple_event(p=p_a)
    if A:
        B = generate_simple_event(p=p_ba)
    else:
        B = generate_simple_event(p=1-p_ba)
    return A, B

complex_event_frequency_function = get_events_frequencies_function(generate_complex_event)

In [6]:
p_a_slider = widgets.FloatSlider(min=0, max=1,step=0.01,value=0.5)
p_ba_slider = widgets.FloatSlider(min=0, max=1,step=0.01,value=0.5)

interact_manual(complex_event_frequency_function,
                p_a=widgets.FloatSlider(min=0, max=1,step=0.01),
                p_ba=widgets.FloatSlider(min=0, max=1,step=0.01))

interactive(children=(FloatSlider(value=0.0, description='p_a', max=1.0, step=0.01), FloatSlider(value=0.0, de…

<function __main__.get_events_frequencies_function.<locals>.events_frequencies(**kwargs)>

### Задание 4

In [7]:
def get_full_group_events_frequencies_function(event_f, n_p):
    def events_frequencies(**kwargs):
        sample_set = [event_f(**kwargs) for _ in range(gen_count)]
        return [sample_set.count(i+1) for i in range(n_p)]
    return events_frequencies

def generate_full_group_event(**kwargs):
    probabilities = [int(p) for p in kwargs['probabilities']]
    prob_sum = sum(probabilities)
    p = random.random()*prob_sum
    thresholds = [sum(probabilities[:i]) for i in range(len(probabilities))]
    for i, threshold in reversed(list(enumerate(thresholds))):
        if p >= threshold:
            return i+1

In [8]:
available_probability = 100
n_p_list = 3
assert 2 <= n_p_list <= 100

full_group_events_frequencies_function = get_full_group_events_frequencies_function(generate_full_group_event, n_p_list)

p_list = [
    [widgets.Label(value='Diff:'),
     widgets.BoundedIntText(value=1, min=1, max=10, step=1, disabled=False),
     widgets.Button(description='Less', disabled=False, tooltip='More', button_style='danger'),
     widgets.Button(description='More', disabled=False, tooltip='More', button_style='success'),
     widgets.Label(value='Probability ->'),
     widgets.Label(value='0')]
    for i in range(n_p_list)
]

def get_handle_less_click_function(diff, label):
    def handle_less_click(b):
        new_label_val = int(label.value) - int(diff.value)
        if new_label_val >= 0:
            label.value = str(new_label_val)
        values = [int(widget_set[5].value) for widget_set in p_list]
        sum_ui.value = (
            f'{sum(values)}'
        )
        res_ui.value = 'Not a full group'
        res2_ui.value = ''
    return handle_less_click

def get_handle_more_click_function(diff, label):
    def handle_more_click(b):
        new_label_val = int(label.value) + int(diff.value)
        if int(sum_ui.value)-int(label.value)+new_label_val <= 100:
            label.value = str(new_label_val)
        values = [int(widget_set[5].value) for widget_set in p_list]
        sum_ui.value = (
            f'{sum(values)}'
        )
        if sum(values) == 100:
            events_frequencies = full_group_events_frequencies_function(probabilities=[widget_set[5].value for widget_set in p_list])
            res_ui.value = f'{events_frequencies}'
            res2_ui.value = f'{[event_frequence/gen_count for event_frequence in events_frequencies]}'
    return handle_more_click
    
# register slides
for widget_set in p_list:
    widget_set[2].on_click(get_handle_less_click_function(widget_set[1], widget_set[5]))
    widget_set[3].on_click(get_handle_more_click_function(widget_set[1], widget_set[5]))

# display ui
widget_set_ui = []
for widget_set in p_list:
    widget_set_ui.append(widgets.HBox(widget_set))
ui = widgets.VBox(widget_set_ui)
sum_ui = widgets.Label(value=f'0')
res_ui = widgets.Label(value=f'Not a full group')
res2_ui = widgets.Label(value='')
display(ui, sum_ui, res_ui, res2_ui)


Label(value='0')

Label(value='Not a full group')

Label(value='')

### Доп. задание

In [9]:
games_and_donations = [('LIMBO', 500),
                       ('Everhood', 1000),
                       ('Undertale', 1500),
                       ('OMORI', 2000)]

donations = [pair[1] for pair in games_and_donations]

print(games_and_donations[generate_full_group_event(probabilities=donations)-1][0])

LIMBO
